## Do the Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt




## Load the csv files

In [3]:
df_categories = pd.read_csv('categories.csv')
df_items = pd.read_csv('items.csv')
df_sales_train = pd.read_csv('sales_train.csv')
df_shops = pd.read_csv('shops.csv')
df_test = pd.read_csv('test.csv')
df_sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
df_sales_train.head()

#df_sales_train[df_sales_train['item_cnt_day'] == -1.0] 

#drop item_cnt_day == -1.0 is the item being returned
#negative item_price is the refund for that product
df_sales_train = df_sales_train[df_sales_train.item_cnt_day != -1.0]

#extract only the data which is from october 
df_sales_train_oct = df_sales_train.drop(df_sales_train.loc[df_sales_train['date_block_num'] != 33 ].index)

df_sales_train_oct

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882335,23.10.2015,33,45,13315,649.0,1.0
2882336,05.10.2015,33,45,13880,229.0,1.0
2882337,02.10.2015,33,45,13881,659.0,1.0
2882338,12.10.2015,33,45,13881,659.0,1.0
2882339,04.10.2015,33,45,13923,169.0,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0


In [10]:
#How many unique shops and items are present in the above data

print(df_sales_train_oct.shop_id.nunique())
print(df_sales_train_oct.item_id.nunique())

#Why these two columns and why not the other?
df_sales_train_oct.groupby(["shop_id", "item_id"])[["item_cnt_day"]].agg(["min", "max", "mean", "count", "sum"])   

#Is this step FE ? 
#So we omitted the rest of the features/columns ?
df_sales_agg = df_sales_train_oct.groupby(["shop_id", "item_id"])["item_cnt_day"].sum().reset_index()
df_sales_agg

44
5411


,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0
...,...,...,...
31516,59,22087,6.0
31517,59,22088,2.0
31518,59,22091,1.0
31519,59,22100,1.0


In [14]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [13]:
#Create a baseline model by taking the values from the test and merging them
#Whats happening here?

results = pd.merge(left = df_test,              # Left table for the join
                   right = df_sales_agg,        # Right table for the join
                   on=["shop_id", "item_id"],   # Common keys
                   how='left')                  # Type of join

results.head()
#results.tail()

,ID,shop_id,item_id,item_cnt_day
0,0,5,5037,NaN
1,1,5,5320,NaN
2,2,5,5233,1.0
3,3,5,5232,NaN
4,4,5,5268,NaN


In [16]:
print("% of missings:", results.item_cnt_day.isna().sum() / len(results) * 100)

#fill the missing nans with zeros
#But aren't we creating noise by simply repalcing nan s with zeros ?
results.item_cnt_day = results.item_cnt_day.fillna(0)
results.head()

% of missings: 86.61437908496733


,ID,shop_id,item_id,item_cnt_day
0,0,5,5037,0.0
1,1,5,5320,0.0
2,2,5,5233,1.0
3,3,5,5232,0.0
4,4,5,5268,0.0


In [18]:
#what are we doing here ?
df_sample_submission["item_cnt_month"] = results["item_cnt_day"]
df_sample_submission.head()

df_sample_submission.to_csv("oct2015.csv")